In [1]:
from __future__ import division
import os,time,cv2,scipy.io
import tensorflow as tf
# import tensorflow.contrib.slim as slim
import tf_slim as slim

import numpy as np
import matplotlib.pyplot as plt
# from discriminator import build_discriminator
import scipy.stats as st

In [2]:
# ENVIRONMENT
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import pathlib
from glob import glob

import warnings
warnings.filterwarnings("ignore")

# TENSORFLOW 2.0
import tensorflow as tf
print(tf.__version__)
print('GPU available:', tf.test.is_gpu_available())

from tensorflow.keras.models import Model
from tensorflow.python.keras.layers import Add, BatchNormalization, Conv2D, Dense, Flatten, Input, LeakyReLU, PReLU, Lambda, MaxPool2D
from tensorflow.keras.optimizers.schedules import PiecewiseConstantDecay
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy, MeanAbsoluteError, MeanSquaredError
binary_cross_entropy = BinaryCrossentropy()

from tensorflow.keras.metrics import Mean

import tensorflow.keras.backend as K
from tensorflow.python.data.experimental import AUTOTUNE

# ESSENTIAL 
import numpy as np
from sklearn.model_selection import train_test_split

# VISUALIZER
import matplotlib.pyplot as plt
plt.ioff()
from IPython import display

# UTILS
import time
from datetime import date
from tqdm import tqdm

2.1.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU available: False


In [6]:
import mahotas 
import imutils

import matplotlib.pyplot as plt
%matplotlib inline

In [22]:
pwd

'/home/jovyan/work/FinalProject'

In [26]:
trans_test = cv2.imread('./Test-Images/back_2.jpg', -1)
blend_test = cv2.imread('./Test-Images/blended_2.jpg', -1)

In [27]:
trans_test.shape, blend_test.shape

((1265, 1043, 3), (1265, 1043, 3))

In [42]:
neww = 512
newh = 512

channel = 64

In [33]:
output_t=cv2.resize(np.float32(trans_test),(neww,newh),cv2.INTER_CUBIC)/255.0
output_b=cv2.resize(np.float32(blend_test),(neww,newh),cv2.INTER_CUBIC)/255.0

In [35]:
# plt.figure(figsize=(12,8))
# plt.imshow(imutils.opencv2matplotlib(output_t))
# plt.show()

# plt.figure(figsize=(12,8))
# plt.imshow(imutils.opencv2matplotlib(output_b))
# plt.show()

In [38]:
output_t = np.expand_dims(output_t,axis=0)
output_b = np.expand_dims(output_b,axis=0)

In [43]:
x_input = tf.concat([output_t, output_b], axis=3)

In [45]:
in_channels = x_input.get_shape()[3]
out_channels = channel

x_filter = tf.compat.v1.get_variable("filter", [4, 4, in_channels, out_channels], dtype=tf.float32, 
                                    initializer=tf.compat.v1.random_normal_initializer(0, 0.02))
# KERNEL_SIZE = 4

In [53]:
in_channels, out_channels, x_filter.shape

(6, 64, TensorShape([4, 4, 6, 64]))

In [52]:
padded_input = tf.pad(tensor=x_input, paddings=[[0, 0], [1, 1], [1, 1], [0, 0]], mode="CONSTANT")
padded_input.shape

TensorShape([1, 514, 514, 6])

In [80]:
padded = np.array([[0, 0], [1, 1], [1, 1], [0, 0]])
padded[0,:]

array([0, 0])

In [190]:
class Discriminator(object):
    def __init__(self):
        self.filters = 64
        self.init_kernel = tf.initializers.he_normal(seed= None)
        self.initializer = tf.random_normal_initializer(0., 0.02)
    
    def lrelu(self, x_in, a):
        x = tf.identity(x_in)
        return (0.5 * (1 + a)) * x + (0.5 * (1 - a)) * tf.abs(x)
    
    def conv_block(self, x_in, filters, strides, batch_norm= True, relu_act= True):
#         x = tf.keras.layers.Lambda(lambda x: tf.pad(x_in, [[0, 0], [1, 1], [1, 1], [0, 0]], mode="CONSTANT"))
        x_in = tf.pad(x_in, [[0, 0], [1, 1], [1, 1], [0, 0]], mode="CONSTANT")
        x = tf.keras.layers.Conv2D(filters= filters, kernel_size= 4,
                                   strides= strides, use_bias= False, 
                                   padding="VALID",
#                                    padding="SAME", 
#                                    kernel_initializer= self.init_kernel,
                                   kernel_initializer= self.initializer)(x_in)
        if batch_norm:
            x = BatchNormalization()(x)
        if relu_act:
            x = self.lrelu(x, 0.2)
#             x = LeakyReLU(0.2)(x)
        return x

#     # with build(input_shape):
#     def build(self, input_shape):
#         inp = tf.keras.layers.Input(shape=input_shape, name='input_image')
#         tar = tf.keras.layers.Input(shape=input_shape, name='target_image')

#         x_in = tf.keras.layers.concatenate([inp, tar])
    
    # with build(discrim_inputs, discrim_targets):
    def build(self, discrim_inputs, discrim_targets):
        input_shape = tf.keras.layers.concatenate([discrim_inputs, discrim_targets]).shape
        
        x_in = tf.keras.layers.Input(shape=input_shape)
        
    # Discriminator Layers:     
        x = self.conv_block(x_in, self.filters, 2, False)  #Layer 1, filters = 64        
#         x = Conv2D(64, 4, strides= 2, use_bias= False, 
#                                    padding="SAME", 
# #                                    kernel_initializer= self.init_kernel,
#                                    kernel_initializer= self.initializer)(x_in)
        
        x = self.conv_block(x, self.filters *2, 2) #Layer 2, filters = 128
        x = self.conv_block(x, self.filters *4, 2) #Layer 3, filters = 256
        x = self.conv_block(x, self.filters *8, 1) #Layer 4, filters = 512  
        x = self.conv_block(x, 1, 1, False, False) #Layer 5
        
        x = tf.keras.activations.sigmoid(x)
#         x = tf.keras.layers.Dense(1, activation='sigmoid')(x)
        
        x = Model(x_in, x)
        return x


In [156]:
test_input = np.float32(np.random.rand(256, 256, 3))
test_target = np.float32(np.random.rand(256, 256, 3))

# test_input  = np.expand_dims(test_input, axis=0)
# test_target = np.expand_dims(test_target, axis=0)

In [108]:
test_input.shape, test_target.shape

x_test = tf.keras.layers.concatenate([test_input, test_target])
x_test.shape

TensorShape([256, 256, 6])

In [191]:
# discriminator = Discriminator().build(input_shape=(256, 256, 3))

discriminator = Discriminator().build(test_input, test_target)

In [192]:
discriminator.summary()

Model: "model_32"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_34 (InputLayer)           [(None, 256, 256, 6) 0                                            
__________________________________________________________________________________________________
tf_op_layer_Pad_15 (TensorFlowO [(None, 258, 258, 6) 0           input_34[0][0]                   
__________________________________________________________________________________________________
conv2d_169 (Conv2D)             (None, 128, 128, 64) 6144        tf_op_layer_Pad_15[0][0]         
__________________________________________________________________________________________________
tf_op_layer_Identity_125 (Tenso [(None, 128, 128, 64 0           conv2d_169[0][0]                 
___________________________________________________________________________________________

In [166]:
# discriminator([s, v])